In [9]:
#pip install unidecode

In [10]:
from sklearn.feature_selection import SelectKBest, f_regression, f_classif
import pandas as pd
import numpy as np
import random
import warnings
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.feature_selection import RFE
from sklearn.decomposition import PCA
from sklearn.metrics import roc_auc_score, mean_squared_error, r2_score
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import cross_val_score
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, roc_auc_score, mean_squared_error, r2_score, confusion_matrix
from sklearn.utils import check_array
from unidecode import unidecode
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.linear_model import ElasticNet
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_percentage_error
from sklearn.linear_model import LogisticRegression
import xgboost as xgb
from imblearn.over_sampling import RandomOverSampler
from sklearn.model_selection import learning_curve



In [8]:
def balancear_datos(X, Y):
    # Instancia el RandomOverSampler
    ros = RandomOverSampler(random_state=13)

    # Aplica la técnica de sobremuestreo para balancear las clases
    X_resampled, Y_resampled = ros.fit_resample(X, Y)

    return X_resampled, Y_resampled
print('balancear_datos Loaded')

balancear_datos Loaded


In [ ]:
# SelectKBest : Selecciona las variables Relevantes Utilizando SelectKBest

def Selectkbest_features(X, y, k,tipo_modelo=f_regression):
    selector = SelectKBest(score_func=f_regression, k=k)  # Utilizamos f_regression como función de puntuación
    X_new = selector.fit_transform(X, y)  # Seleccionamos las k mejores características

    mask = selector.get_support()  # Máscara booleana de las características seleccionadas
    selected_features = [feature for feature, mask_value in zip(X.columns, mask) if mask_value]
    selected_scores = selector.scores_[selector.get_support()]

    # Crear un DataFrame con las variables seleccionadas y sus puntuaciones
    selected_data = pd.DataFrame({'Variable': selected_features, 'Puntuacion': selected_scores})

#f_regression; f_classif
    return selected_data
print('Selectkbest_features Loaded')

Selectkbest_features Loaded


In [ ]:
# RFE: Fucnion que obtiene las variables utilizanfo el modelo de Recursive Feature Elimination.

def rfe_features(estimator, X, y, n_features_to_select=None):

    warnings.filterwarnings("ignore", category=DeprecationWarning)
    selector = RFE(estimator, n_features_to_select=n_features_to_select)
    selector.fit(X, y)

    # Obtener las variables seleccionadas
    selected_features = X.columns[selector.support_].tolist()


    # Obtener los puntajes de las características seleccionadas
    feature_scores = selector.estimator_.coef_

    # Crear el DataFrame de resultado
    df_result = pd.DataFrame({'Variable': selected_features, 'Score': feature_scores})
    warnings.filterwarnings("default", category=DeprecationWarning)
    return df_result
print('rfe_features Loaded')

rfe_features Loaded


In [ ]:
 # Integración de Variables

def integrate_features(Selectkbest_List,Boruta_List,Rfe_List):

    integrated_features = list(set(Selectkbest_List + Boruta_List + Rfe_List))

    return integrated_features
print('Integración de Variables Loaded')

Integración de Variables Loaded


In [ ]:
# Funcion de evaluacion de los modelos

def evaluate_results(y_true, y_pred):
    # Curva ROC
    roc_auc = roc_auc_score(y_true, y_pred)

    # Error cuadrático medio (MSE)
    mse = mean_squared_error(y_true, y_pred)

    # Raíz del error cuadrático medio (RMSE)
    rmse = np.sqrt(mse)

    # Coeficiente de determinación (R2)
    r2 = r2_score(y_true, y_pred)

    # Error porcentual absoluto medio (MAPE)
    mape = np.mean(np.abs((y_true - y_pred) / y_true)) * 100

    return roc_auc, mse, rmse, r2, mape
print('evaluacion de los modelos Loaded')

In [ ]:
# Generacion de Componentes Principales

def generar_componentes_principales_X(dataframe, n_componentes):
    # Inicializar el objeto PCA con el número de componentes deseados
    pca = PCA(n_components=n_componentes)

    # Realizar el PCA y transformar los datos
    componentes_principales = pca.fit_transform(dataframe)

    # Crear un DataFrame con los componentes principales
    df_componentes = pd.DataFrame(data=componentes_principales, columns=[f'Componente_{i+1}' for i in range(n_componentes)])

    return df_componentes
print('Pca_components Loaded')

Pca_components Loaded


In [ ]:
#Funcion que encuentra las variables utilziando tensorflow

def find_important_variables(X, y, model_type='random_forest', n_estimators=100):
    # Configurar TensorFlow para utilizar la GPU
    physical_devices = tf.config.list_physical_devices('GPU')
    tf.config.experimental.set_memory_growth(physical_devices[0], True)

    if model_type == 'random_forest':
        # Crear el modelo Random Forest
        model = RandomForestRegressor(n_estimators=n_estimators)

    elif model_type == 'xgboost':
        # Crear el modelo XGBoost
        model = XGBRegressor(n_estimators=n_estimators)

    else:
        raise ValueError('Model type not supported.')

    # Convertir los datos a tensores de TensorFlow
    X_tensor = tf.convert_to_tensor(X.values, dtype=tf.float32)
    y_tensor = tf.convert_to_tensor(y.values, dtype=tf.float32)

    # Entrenar el modelo
    with tf.device('/GPU:0'):
        model.fit(X_tensor, y_tensor)

    # Obtener la importancia de las variables
    if model_type == 'random_forest':
        feature_importances = model.feature_importances_
    elif model_type == 'xgboost':
        feature_importances = model.feature_importances_
    else:
        feature_importances = None

    if feature_importances is not None:
        # Obtener los nombres de las variables
        variable_names = X.columns

        # Crear un DataFrame con las variables y su importancia
        importance_df = pd.DataFrame({'Variable': variable_names, 'Importance': feature_importances})

        # Ordenar las variables por su importancia en orden descendente
        importance_df = importance_df.sort_values('Importance', ascending=False).reset_index(drop=True)

        return importance_df
    else:
        raise ValueError('Feature importances not available.')
print('Improtancia con XGBoost y Random forest Loaded')

In [ ]:
#Modelo para encontrar variables importantes usando XGBOOST y RANDOM FOREST
def find_important_variables_Clasificacion(X, y, model_type='xgboost', n_estimators=100):
    if model_type == 'xgboost':
        # Crear el modelo XGBoost clasificador
        model = xgb.XGBClassifier(n_estimators=n_estimators)

    elif model_type == 'random_forest':
        # Crear el modelo Random Forest clasificador
        model = RandomForestClassifier(n_estimators=n_estimators, random_state=42)

    else:
        raise ValueError('Model type not supported. Use "xgboost" or "random_forest".')


    # Dividir los datos en conjuntos de entrenamiento y prueba
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Entrenar el modelo
    model.fit(X_train, y_train)

    # Obtener la importancia de las variables
    feature_importances = model.feature_importances_

    # Obtener los nombres de las variables
    variable_names = X.columns

    # Crear un DataFrame con las variables y su importancia
    importance_df = pd.DataFrame({'Variable': variable_names, 'Importance': feature_importances})

    # Ordenar las variables por su importancia en orden descendente
    importance_df = importance_df.sort_values('Importance', ascending=False).reset_index(drop=True)

    return importance_df

In [ ]:
def xgboost_model_evaluation(X_train, X_test, y_train, y_test):
    # Definir los hiperparámetros a ajustar
    param_grid = {
        'max_depth': [3, 5, 7],
        'learning_rate': [0.1, 0.01, 0.001],
        'n_estimators': [100, 500, 1000]
    }

    # Crear el modelo XGBoost
    xgb_model = XGBRegressor()

    # Realizar Grid Search con validación cruzada
    grid_search = GridSearchCV(estimator=xgb_model, param_grid=param_grid, scoring='neg_mean_squared_error', cv=5)
    grid_search.fit(X_train, y_train)

    # Obtener los mejores hiperparámetros
    best_params = grid_search.best_params_

    # Crear el modelo XGBoost con los mejores hiperparámetros
    xgb_best_model = XGBRegressor(**best_params)

    # Ajustar el modelo con los mejores hiperparámetros a los datos de entrenamiento
    xgb_best_model.fit(X_train, y_train)

    # Realizar la predicción en el grupo de prueba
    y_pred = xgb_best_model.predict(X_test)

    # Calcular el error cuadrático medio (MSE) del modelo en el grupo de prueba
    mse = mean_squared_error(y_test, y_pred)

    # Calcular el coeficiente de determinación (R2) del modelo en el grupo de prueba
    r2 = r2_score(y_test, y_pred)

    # Calcular el error porcentual absoluto medio (MAPE) del modelo en el grupo de prueba
    mape = mean_absolute_percentage_error(y_test, y_pred) * 100

    return y_pred, best_params, mse, r2, mape
print('XG_BOOST Regression Loaded')

XG_BOOST Regression Loaded


In [ ]:
def random_forest_model_evaluation(X_train, X_test, y_train, y_test):
    """
    Genera la predicción y evaluación de un modelo RandomForestRegressor utilizando Grid Search para encontrar los mejores hiperparámetros.

    Parámetros:
    - X_train: matriz de características para el grupo de entrenamiento.
    - X_test: matriz de características para el grupo de prueba.
    - y_train: vector de etiquetas para el grupo de entrenamiento.
    - y_test: vector de etiquetas para el grupo de prueba.

    Retorna:
    - y_pred: predicciones del modelo en el grupo de prueba.
    - best_params: mejores hiperparámetros encontrados mediante Grid Search.
    - accuracy: exactitud (accuracy) del modelo en el grupo de prueba.
    """
    # Definir los hiperparámetros a ajustar
    param_grid = {
        'n_estimators': [100, 200, 500],
        'max_depth': [None, 5, 10],
        'min_samples_split': [2, 5, 10],
        'min_samples_leaf': [1, 2, 4]
    }

    # Crear el modelo RandomForestRegressor
    rf_model = RandomForestRegressor()

    # Realizar Grid Search con validación cruzada
    grid_search = GridSearchCV(estimator=rf_model, param_grid=param_grid, scoring='accuracy', cv=5)
    grid_search.fit(X_train, y_train)

    # Obtener los mejores hiperparámetros
    best_params = grid_search.best_params_

    # Crear el modelo RandomForestRegressor con los mejores hiperparámetros
    rf_best_model = RandomForestRegressor(**best_params)

    # Ajustar el modelo con los mejores hiperparámetros a los datos de entrenamiento
    rf_best_model.fit(X_train, y_train)

    # Realizar la predicción en el grupo de prueba
    y_pred = rf_best_model.predict(X_test)

    # Calcular la exactitud del modelo en el grupo de prueba
    accuracy = accuracy_score(y_test, y_pred)

    return y_pred, best_params, accuracy
print('Random Forest Loaded')

Random Forest Loaded


In [ ]:
##Función que elimina las tildes y diacríticos de un texto.
def quitar_tildes(texto):
    texto_sin_tildes = unidecode(texto)
    return texto_sin_tildes
print('quitar_tildes Loaded')

quitar_tildes Loaded


In [ ]:
##Función completa datos faltantes de serie de timepo
def completar_datos_serie_tiempo(df,inicio,fin):

    # Convert 'Year' to datetime type with date set to December 31st
    df['Year'] = pd.to_datetime(df['Year'].astype(str) + '-12-31')

    # Set 'Year' and 'Category' as the index
    df.set_index(['Year', 'Category'], inplace=True)

    # Create a new DataFrame with all years and categories from the first year to the last year in the available data
    start_year = inicio
    end_year = fin
    all_years = pd.DataFrame(index=pd.MultiIndex.from_product([[str(y) + '-12-31' for y in range(start_year, end_year+1)], df.index.get_level_values('Category').unique()], names=['Year', 'Category']))


    all_years['join']=all_years.index.get_level_values('Year').astype(str) + '-' + all_years.index.get_level_values('Category').astype(str)
    df['join']=df.index.get_level_values('Year').astype(str) + '-' + df.index.get_level_values('Category').astype(str)


    # Join the existing DataFrame with all_years
    all_years.reset_index(inplace=True)
    all_years['Year'] = pd.to_datetime(all_years['Year'])
    df_complete = pd.merge(all_years, df,on='join',how='left')
    #df_complete_s=df_complete
    df_complete.set_index(['Year', 'Category'], inplace=True)
    df_complete = df_complete.drop('join', axis=1)


    # Obtener la lista de columnas que necesitan ser completadas
    columns_to_interpolate = df_complete.columns.tolist()


    for column in columns_to_interpolate:
        df_complete[column] = df_complete.groupby('Category',group_keys=False)[column].apply(lambda x: x.interpolate(limit_direction='both'))

    df_complete.reset_index(inplace=True)
    df_complete=df_complete.fillna(0)
    df_complete['Year'] = df_complete['Year'].dt.year


    return df_complete
print('completar_datos_serie_tiempo Loaded')

completar_datos_serie_tiempo Loaded


In [ ]:
#elimina las filas con NAN
def eliminar_filas_nan(dataframe, columna):
    dataframe_sin_nan = dataframe.dropna(subset=[columna], inplace=False)
    return dataframe_sin_nan
print('eliminar_filas_nan Loaded')

eliminar_filas_nan Loaded


In [ ]:
#si encuentra numeros menores los convierte en Nan
def cambiar_numeros_menores(dataframe, valor, columna_excluida):
    for columna in dataframe.columns:
        if columna != columna_excluida:
            dataframe[columna] = np.where(dataframe[columna] < valor, np.nan, dataframe[columna])
    return dataframe
print('cambiar_numeros_menores Loaded')

cambiar_numeros_menores Loaded


In [ ]:
#Funcion que Normaliza variables numericas

def normalize_variables(dataset):
    scaler = MinMaxScaler()
    normalized_dataset = scaler.fit_transform(dataset)
    normalized_dataset = pd.DataFrame(normalized_dataset, columns=dataset.columns)
    return normalized_dataset
print('normalize_variables Loaded')

normalize_variables Loaded


In [ ]:
#Calcula el crecimiento respecto a un año especifico
def calcular_crecimiento_ingresos(data, years_offset,Columna,ingresos,Yearp):
    data_sorted = data.sort_values(by=[Yearp])
    data_pivot = data_sorted.pivot(index=Columna, columns=Yearp, values=ingresos)

    # Obtener todas las columnas de años en el DataFrame
    years = data_pivot.columns.tolist()

    crecimiento_data = []

    if years_offset < 0:
        years_offset = abs(years_offset)
        for i in range(years_offset, len(years)):
            current_year = years[i]
            past_year = years[i - years_offset]

            crecimiento = (data_pivot[current_year] - data_pivot[past_year]) / data_pivot[past_year]

            crecimiento_data.append(pd.DataFrame({Columna: data_pivot.index, Yearp: current_year, 'Crecimiento': crecimiento}))
    else:
        for i in range(len(years) - years_offset):
            current_year = years[i]
            future_year = years[i + years_offset]

            crecimiento = (data_pivot[future_year] - data_pivot[current_year]) / data_pivot[current_year]

            crecimiento_data.append(pd.DataFrame({Columna: data_pivot.index, Yearp: current_year, 'Crecimiento': crecimiento}))

    result = pd.concat(crecimiento_data, ignore_index=True)

    return result
print('calcular_crecimiento_ingresos Loaded')

calcular_crecimiento_ingresos Loaded


In [ ]:
# Genera grupos de variables que comparten alta correlacion
def obtener_variables_correlacionadas(data, umbral):
    # Calcular la matriz de correlación
    matriz_correlacion = data.corr().abs()

    # Crear un diccionario para mapear las variables con sus grupos
    grupos = {}
    num_grupo = 1

    # Recorrer las variables y asignarles el número de grupo
    for variable in matriz_correlacion.columns:
        if variable not in grupos:
            variables_correlacionadas = matriz_correlacion.index[matriz_correlacion[variable] >= umbral]
            for var in variables_correlacionadas:
                grupos[var] = num_grupo
            num_grupo += 1

    # Crear un DataFrame con las variables agrupadas
    df_variables_correlacionadas = pd.DataFrame({'Variable': list(grupos.keys()), 'Grupo': list(grupos.values())})

    return df_variables_correlacionadas
print('obtener_variables_correlacionadas Loaded')

obtener_variables_correlacionadas Loaded


In [ ]:

def generar_componentes_principales(df_datos, df_grupos):
    grupos = df_grupos['Grupo'].unique()  # Obtener la lista de grupos únicos
    componentes_p = pd.DataFrame()
    for grupo in grupos:
        # Filtrar las variables correspondientes al grupo actual
        variables_grupo = df_grupos[df_grupos['Grupo'] == grupo]['Variable'].tolist()

        # Filtrar las columnas del DataFrame de datos según las variables del grupo actual
        datos_grupo = df_datos[variables_grupo]

        # Aplicar PCA al DataFrame del grupo actual
        pca = PCA(n_components=1)
        componente_principal = pca.fit_transform(datos_grupo)

        # Asignar nombre al componente principal
        nombre_componente = "Componente_Principal_" + str(grupo)
        df_temp = pd.DataFrame(componente_principal, columns=[nombre_componente])
        componentes_p = pd.concat([componentes_p, df_temp], axis=1)
        # Agregar el componente principal al DataFrame de datos

    return componentes_p
print('generar_componentes_principales Loaded')

generar_componentes_principales Loaded


In [ ]:
# RFE: Fucnion que obtiene las variables utilizanfo el modelo de Recursive Feature Elimination.

def rfe_with_elasticnet(X, y, n_features_to_select):
    # Crear el estimador ElasticNet
    estimator = ElasticNet()

    # Crear el objeto RFE
    rfe = RFE(estimator=estimator, n_features_to_select=n_features_to_select)

    # Realizar la selección de características
    rfe.fit(X, y)

    # Obtener los nombres de las características
    feature_names = X.columns

    # Obtener los coeficientes de importancia de las características
    feature_importances = rfe.estimator_.coef_

    # Crear un DataFrame con las características y sus coeficientes de importancia
    result_df = pd.DataFrame({'Feature': feature_names, 'Importance': feature_importances})

    return result_df
print('rfe_with_elasticnet Loaded')

In [ ]:
# Funcion para calcular RFE En classificación

def select_features_with_rfe_Clasisification(X, y, num_features):
    # Instanciar el modelo de clasificación (por ejemplo, Regresión Logística)
    model = LogisticRegression(max_iter=1000)

    # Crear el objeto RFE y especificar el modelo y el número de características deseado
    rfe = RFE(model, n_features_to_select=num_features)

    # Ajustar RFE a los datos
    rfe.fit(X, y)

    # Obtener las características seleccionadas con sus coeficientes de importancia
    selected_features = X.columns[rfe.support_]
    coef_importance = rfe.estimator_.coef_[0]

    # Crear un DataFrame con las características seleccionadas y sus coeficientes de importancia
    result_df = pd.DataFrame({'Variable': selected_features, 'Importance': coef_importance})

    return result_df

In [ ]:
def calcular_crecimiento_promedio_por_categoria_Predial(df, Year_inicio_periodo1, Year_fin_periodo1, Year_inicio_periodo2, Year_fin_periodo2):
    # Filtrar el DataFrame por los años de cada periodo
    periodo1 = df[(df['Año'] >= Year_inicio_periodo1) & (df['Año'] <= Year_fin_periodo1)]
    periodo2 = df[(df['Año'] >= Year_inicio_periodo2) & (df['Año'] <= Year_fin_periodo2)]

    # Calcular el promedio de valores para cada categoría en cada periodo
    promedio_periodo1 = periodo1.groupby('CODIGO_DANE')['Predial'].mean()
    promedio_periodo2 = periodo2.groupby('CODIGO_DANE')['Predial'].mean()

    # Calcular el crecimiento porcentual promedio para cada categoría
    crecimiento_promedio = ((promedio_periodo2 - promedio_periodo1) / promedio_periodo1) * 100

    return crecimiento_promedio
print('calcular_crecimiento_promedio_por_categoria_Predial Loaded')

In [ ]:
def categorizar_por_boxplot(df,columna_valores):
    # Calcular el primer, segundo y tercer cuartil
    primer_cuartil = columna_valores.quantile(0.25)
    segundo_cuartil = columna_valores.quantile(0.5)
    tercer_cuartil = columna_valores.quantile(0.75)

    # Calcular los límites del bigote inferior y superior
    iqr = tercer_cuartil - primer_cuartil
    limite_inferior = primer_cuartil - 1.5 * iqr
    limite_superior = tercer_cuartil + 1.5 * iqr

    # Categorizar los valores según los análisis de boxplot
    categorias = []
    for valor in columna_valores:
        if valor < limite_inferior:
            categorias.append('Muy baja')
        elif valor < primer_cuartil:
            categorias.append('Baja')
        elif valor < tercer_cuartil:
            categorias.append('Media')
        elif valor <= limite_superior:
            categorias.append('Alta')
        else:
            categorias.append('Muy alta')
    df['recomendacion'] = categorias
    return categorias
print('categorizar_por_boxplot Loaded')

In [ ]:
def calcular_crecimiento_promedio_por_categoria(df, columnas_procesar, Year_inicio_periodo1, Year_fin_periodo1, Year_inicio_periodo2, Year_fin_periodo2):
    # Filtrar el DataFrame por los años de cada periodo
    periodo1 = df[(df['Year'] >= Year_inicio_periodo1) & (df['Year'] <= Year_fin_periodo1)]
    periodo2 = df[(df['Year'] >= Year_inicio_periodo2) & (df['Year'] <= Year_fin_periodo2)]

    # Crear un DataFrame vacío para almacenar los resultados
    resultados = pd.DataFrame()

    # Iterar sobre las columnas y calcular el crecimiento por categoría
    for columna in columnas_procesar:
        # Calcular el promedio de valores para cada categoría en cada periodo
        promedio_periodo1 = periodo1.groupby('Municipio')[columna].mean()
        promedio_periodo2 = periodo2.groupby('Municipio')[columna].mean()

        # Calcular el crecimiento porcentual promedio para cada categoría
        crecimiento_promedio = ((promedio_periodo2 - promedio_periodo1) / promedio_periodo1) * 100

        # Agregar el resultado al DataFrame resultados
        resultados = pd.concat([resultados, crecimiento_promedio], axis=1)

    # Renombrar las columnas con los nombres originales
    resultados.columns = columnas_procesar

    return resultados
print('calcular_crecimiento_promedio_por_categoria Loaded')

In [ ]:
def limitar_atipicos(df, columnas_procesar):
    # Inicializar un DataFrame para almacenar los resultados
    resultados = df.copy()

    # Iterar sobre las columnas a procesar
    for columna in columnas_procesar:
        # Calcular el rango intercuartílico (IQR)
        q1 = df[columna].quantile(0.25)
        q3 = df[columna].quantile(0.75)
        iqr = q3 - q1

        # Calcular los límites del bigote superior e inferior
        upper_limit = q3 + 1.5 * iqr
        lower_limit = q1 - 1.5 * iqr

        # Reemplazar los valores atípicos con los límites del bigote
        resultados[columna] = resultados[columna].apply(lambda x: upper_limit if x > upper_limit else lower_limit if x < lower_limit else x)

    return resultados
print('limitar_atipicos Loaded')

In [ ]:
def crear_dummies(df, columns_to_dummy):
    for col in columns_to_dummy:
        dummies = pd.get_dummies(df[col], prefix=col)
        df = pd.concat([df, dummies], axis=1)
        df.drop(col, axis=1, inplace=True)
    return df
print('crear_dummies Loaded')

In [ ]:
#Funcion para comvertir un data frame con años en columasn en filas y generar comparacion de fechas
def year_columnas(Industria, palabra,ID,Year_inicio_periodo1, Year_fin_periodo1, Year_inicio_periodo2, Year_fin_periodo2):

    new_columns = [col.replace(palabra, '') for col in Industria.columns] # quita un valor de una columna
    Industria.rename(columns=dict(zip(Industria.columns, new_columns)), inplace=True) # quita un valor de una columna

    Industria = Industria.melt(id_vars=[ID], var_name='Year', value_name='Valor') #Transpone los años en filas

    Industria.rename(columns={ID: 'Category'}, inplace=True) #Cambio de nombre de columna
    Industria=completar_datos_serie_tiempo(Industria,2008,2020) #Completa faltantes con interpolación

    Industria.rename(columns={'Category': 'Municipio'}, inplace=True) #Cambio de nombre de columna
    Industria.rename(columns={'Valor': 'Industria'}, inplace=True) #Cambio de nombre de columna
    lista_columnas = Industria[Industria.columns.difference(['Municipio','Year'])].columns.tolist()
    Industria=calcular_crecimiento_promedio_por_categoria(Industria,lista_columnas,Year_inicio_periodo1, Year_fin_periodo1, Year_inicio_periodo2, Year_fin_periodo2)
    Industria=limitar_atipicos(Industria, lista_columnas)
    Industria = Industria.fillna(0)
    Industria=Industria.reset_index()
    Industria.rename(columns={'index': 'Municipio'}, inplace=True) #Cambio de nombre de columna
    return Industria

In [ ]:
def train_and_evaluate_model(X_train, X_test, y_train, y_test, Modelo, param_grid=None, random_state=13):
    if Modelo not in ['xgboost', 'random_forest']:
        raise ValueError('Modelo debe ser "xgboost" o "random_forest".')

    if Modelo == 'xgboost':
        model = xgb.XGBClassifier(tree_method='gpu_hist', random_state=random_state)
        model = xgb.XGBClassifier(random_state=random_state)
    else:
        model = RandomForestClassifier(random_state=random_state)

    if param_grid is not None:
        grid_search = GridSearchCV(estimator=model, param_grid=param_grid, scoring='roc_auc_ovr', cv=5, n_jobs=-1)
        grid_search.fit(X_train, y_train)
        best_params = grid_search.best_params_
        model.set_params(**best_params)

    # Validación cruzada (k-fold cross-validation)
    cv_scores = cross_val_score(model, X_train, y_train, cv=5, scoring='roc_auc_ovr', n_jobs=-1)
    print("Mean CV ROC AUC:", cv_scores.mean())

    # Curva de aprendizaje
    #train_sizes, train_scores, test_scores = learning_curve(model, X_train, y_train, cv=5, scoring='roc_auc_ovr', n_jobs=-1)
    train_sizes, train_scores, test_scores = learning_curve(model, X_train, y_train, cv=5, scoring='roc_auc_ovr', n_jobs=-1)
    plt.figure()
    plt.plot(train_sizes, train_scores.mean(axis=1), 'o-', label='Training Score')
    plt.plot(train_sizes, test_scores.mean(axis=1), 'o-', label='Test Score')
    plt.xlabel('Training examples')
    plt.ylabel('ROC AUC')
    plt.legend(loc='best')
    plt.show()

    with tf.device('/GPU:0'):
        model.fit(X_train, y_train)

    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    roc_auc = roc_auc_score(y_test, model.predict_proba(X_test), multi_class='ovr')
    conf_matrix = confusion_matrix(y_test, y_pred)

    return model, accuracy, roc_auc, conf_matrix, best_params

In [ ]:
def evaluate_model(model, X_eval, y_eval):
    # Obtener las predicciones del modelo
    y_pred = model.predict(X_eval)

    # Calcular el accuracy
    accuracy = accuracy_score(y_eval, y_pred)

    # Calcular el valor de la curva ROC
    #roc_auc = roc_auc_score(y_eval, y_pred,multi_class='ovr')
    roc_auc = roc_auc_score(y_eval, model.predict_proba(X_eval), multi_class='ovr')
    # Calcular la matriz de confusión
    conf_matrix = confusion_matrix(y_eval, y_pred)

    # Crear el DataFrame con los resultados
    result_df = pd.DataFrame({
        'Accuracy': [accuracy],
        'ROC_AUC': [roc_auc]
    })

    return result_df,conf_matrix

In [ ]:
def obtener_clasificacion_y_scores(modelo, X, lista_id):
    """
    Obtiene la clasificación de y y el score de cada categoría para un conjunto de datos X.

    Parámetros:
        modelo: Modelo entrenado capaz de realizar predicciones multiclase.
        X: Conjunto de datos de entrada para realizar las predicciones.
        lista_id (list): Lista de identificadores (ID) correspondientes a los datos en X.

    Retorna:
        pandas DataFrame: Un DataFrame con las columnas ID, y, y las scores para cada categoría.
    """
    # Realizar predicciones para el conjunto de datos X
    y_pred_prob = modelo.predict_proba(X)

    # Obtener las categorías del modelo
    categorias = modelo.classes_

    # Crear una lista con el resultado para cada fila
    resultados = []
    for i in range(len(X)):
        fila = [lista_id[i]] + [categorias[np.argmax(y_pred_prob[i])]] + y_pred_prob[i].tolist()
        resultados.append(fila)

    # Crear el DataFrame con los resultados
    columnas = ['ID', 'y'] + [f'Score_{categoria}' for categoria in categorias]
    df_resultados = pd.DataFrame(resultados, columns=columnas)

    return df_resultados